In [1]:
import pandas as pd
import requests

## Overall Feed

In [2]:
r = requests.get("https://gbfs.citibikenyc.com/gbfs/2.3/gbfs.json")

In [3]:
feed_data = r.json()

In [4]:
feed_data

{'data': {'en': {'feeds': [{'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/gbfs.json',
     'name': 'gbfs'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_information.json',
     'name': 'system_information'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_information.json',
     'name': 'station_information'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json',
     'name': 'station_status'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/free_bike_status.json',
     'name': 'free_bike_status'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_hours.json',
     'name': 'system_hours'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_calendar.json',
     'name': 'system_calendar'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_regions.json',
     'name': 'system_regions'},
    {'url': 'https://gbfs.lyft.com/gbfs/2.3/bkn/en/system_pricing_plans.json',
     'name': 'system_pricing_plans'},
    {'url': 'https://gb

## Station Status

In [5]:
station_r = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/en/station_status.json")

In [6]:
station_df = pd.DataFrame.from_dict(station_r.json().get('data').get('stations'))

In [8]:
station_df.head()

,num_docks_disabled,num_bikes_available,vehicle_types_available,station_id,num_scooters_available,num_ebikes_available,num_scooters_unavailable,num_bikes_disabled,is_renting,num_docks_available,last_reported,is_installed,is_returning
0,0,10,"[{'count': 9, 'vehicle_type_id': '1'}, {'count...",7f4753d7-8d58-48bd-a881-de5bff08b27d,0.0,1,0.0,2,1,14,1730478873,1,1
1,0,19,"[{'count': 7, 'vehicle_type_id': '1'}, {'count...",69717638-5c4a-47a7-bccb-3b42c81eb09f,0.0,12,0.0,0,1,0,1730478867,1,1
2,0,9,"[{'count': 8, 'vehicle_type_id': '1'}, {'count...",1835618878584115330,0.0,1,0.0,4,1,11,1730478868,1,1
3,0,24,"[{'count': 6, 'vehicle_type_id': '1'}, {'count...",66dd01c5-0aca-11e7-82f6-3863bb44ef7c,0.0,18,0.0,2,1,0,1730478870,1,1
4,0,16,"[{'count': 14, 'vehicle_type_id': '1'}, {'coun...",579e583e-60ab-4f2a-83d0-fabebddfcd0f,0.0,2,0.0,1,1,2,1730478870,1,1


In [9]:
station_df["vehicle_types_available"][1]

[{'count': 7, 'vehicle_type_id': '1'}, {'count': 12, 'vehicle_type_id': '2'}]

### Side Note

What the heck do those `vehicle_type_id`s even mean? Look here: https://gbfs.lyft.com/gbfs/2.3/bkn/es/vehicle_types.json

In [10]:
r_vehicle_types = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/es/vehicle_types.json")

In [11]:
r_vehicle_types.json()

{'data': {'vehicle_types': [{'vehicle_type_id': '1',
    'form_factor': 'bicycle',
    'propulsion_type': 'human'},
   {'vehicle_type_id': '2',
    'max_range_meters': 54717.56,
    'form_factor': 'bicycle',
    'propulsion_type': 'electric_assist'}]},
 'last_updated': 1730479128,
 'ttl': 60,
 'version': '2.3'}

In [12]:
# filter the list of vehicle types available for the given id
def filter_by_vehicle_type(count_list, target_id):
    return next((item["count"] for item in count_list if item["vehicle_type_id"] == str(target_id)), 0)

In [13]:
station_df["electric_bikes_available"] = station_df["vehicle_types_available"].apply(lambda x: filter_by_vehicle_type(x, 2))
station_df["regular_bikes_available"] = station_df["vehicle_types_available"].apply(lambda x: filter_by_vehicle_type(x, 1))

In [14]:
station_df

,num_docks_disabled,num_bikes_available,vehicle_types_available,station_id,num_scooters_available,num_ebikes_available,num_scooters_unavailable,num_bikes_disabled,is_renting,num_docks_available,last_reported,is_installed,is_returning,electric_bikes_available,regular_bikes_available
0,0,10,"[{'count': 9, 'vehicle_type_id': '1'}, {'count...",7f4753d7-8d58-48bd-a881-de5bff08b27d,0.0,1,0.0,2,1,14,1730478873,1,1,1,9
1,0,19,"[{'count': 7, 'vehicle_type_id': '1'}, {'count...",69717638-5c4a-47a7-bccb-3b42c81eb09f,0.0,12,0.0,0,1,0,1730478867,1,1,12,7
2,0,9,"[{'count': 8, 'vehicle_type_id': '1'}, {'count...",1835618878584115330,0.0,1,0.0,4,1,11,1730478868,1,1,1,8
3,0,24,"[{'count': 6, 'vehicle_type_id': '1'}, {'count...",66dd01c5-0aca-11e7-82f6-3863bb44ef7c,0.0,18,0.0,2,1,0,1730478870,1,1,18,6
4,0,16,"[{'count': 14, 'vehicle_type_id': '1'}, {'coun...",579e583e-60ab-4f2a-83d0-fabebddfcd0f,0.0,2,0.0,1,1,2,1730478870,1,1,2,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,0,8,"[{'count': 4, 'vehicle_type_id': '1'}, {'count...",66dd5a42-0aca-11e7-82f6-3863bb44ef7c,0.0,4,0.0,1,1,13,1730478931,1,1,4,4
2223,0,5,"[{'count': 5, 'vehicle_type_id': '1'}, {'count...",66dddd28-0aca-11e7-82f6-3863bb44ef7c,0.0,0,0.0,2,1,7,1730478920,1,1,0,5
2224,0,4,"[{'count': 4, 'vehicle_type_id': '1'}, {'count...",66ddd8a8-0aca-11e7-82f6-3863bb44ef7c,0.0,0,0.0,1,1,9,1730478975,1,1,0,4
2225,0,14,"[{'count': 13, 'vehicle_type_id': '1'}, {'coun...",46813ecf-8df4-4c8f-9579-0179e0b36ba6,0.0,1,0.0,1,1,3,1730478979,1,1,1,13


## Station Information

Alright, now we know about the stations, but we need some metadata

In [15]:
r_station_information = requests.get("https://gbfs.lyft.com/gbfs/2.3/bkn/es/station_information.json")

In [16]:
station_metadata = pd.DataFrame.from_dict(r_station_information.json().get('data').get('stations'))

In [17]:
station_metadata.head()

,short_name,rental_uris,station_id,region_id,capacity,lon,name,lat
0,3680.04,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",2c8b263e-952c-4941-a671-ef0ebb062632,71,21,-73.937620,E New York Ave & Troy Ave,40.662270
1,6290.06,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",1846085439406916338,71,24,-73.888200,41 Ave & 77 St,40.745570
2,3246.06,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",1828965172312150882,71,22,-73.946350,New York Ave & Snyder Ave,40.648680
3,8458.03,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",b4974d89-932c-4a9f-a264-2ea0cf70613e,71,20,-73.937878,Pinehurst Ave & W 187 St,40.855331
4,6981.16,"{'ios': 'https://bkn.lft.to/lastmile_qr_scan',...",0630217a-3913-47d9-9540-cf85688296a5,71,19,-73.912094,37 St & 24 Ave,40.770148


## Tie It All Together

In [18]:
all_station_data = pd.merge(
    station_metadata,
    station_df,
    how="left",
    on="station_id"
)

In [19]:
all_station_data.sort_values("electric_bikes_available", ascending=False)[["station_id", "name", "electric_bikes_available", "capacity"]]

,station_id,name,electric_bikes_available,capacity
1486,66dc7b10-0aca-11e7-82f6-3863bb44ef7c,W 51 St & 6 Ave,74,81
287,c638ec67-9ac0-416f-944f-619926144931,E 40 St & Park Ave,60,123
416,1975518133370609774,Broadway & E 19 St,57,99
177,66dc81ce-0aca-11e7-82f6-3863bb44ef7c,E 51 St & Lexington Ave,54,75
576,66dc48ef-0aca-11e7-82f6-3863bb44ef7c,W 44 St & 5 Ave,53,68
...,...,...,...,...
870,5eac53d0-5510-441a-9dbb-76bd38acf4ef,Goulden Ave & W 197 St,0,20
866,9b7e3b8b-97ef-4038-820c-7cd1c1a34fc7,E 116 St & 2 Ave,0,31
1726,d873e463-2dc9-4bf8-86c7-fea8ddaa3a9f,Washington Ave & E 167 St,0,19
1728,35915f48-31c5-499a-b845-2ca2fb541904,Yankee Ferry Terminal,0,42


In [18]:
all_station_data.sort_values("regular_bikes_available", ascending=False)[["station_id", "name", "regular_bikes_available", "capacity"]]

,station_id,name,regular_bikes_available,capacity
1661,1960020817312746312,Allen St & Hester St,75,117
1927,66dc462a-0aca-11e7-82f6-3863bb44ef7c,W 15 St & 7 Ave,71,79
666,66dc7659-0aca-11e7-82f6-3863bb44ef7c,FDR Drive & E 35 St,66,123
1812,66dbf140-0aca-11e7-82f6-3863bb44ef7c,E 2 St & 2 Ave,63,84
69,66de25bd-0aca-11e7-82f6-3863bb44ef7c,3 St & 3 Ave,63,79
...,...,...,...,...
514,2b33334b-243c-4a16-a6d2-32304c8884f1,Popham Ave & W 174 St,0,20
516,e666a344-1afb-4512-b2e9-2fc5ed71be2b,31 St & Ditmars Blvd,0,24
519,db91c9e0-d80b-4cb4-9cc2-9453ec8fbeac,Grand Concourse & E Mount Eden Ave,0,19
1704,42bf7170-61ab-44a0-aafe-1b89d7271718,Prospect Park SW & 10 Ave,0,30


## What do you want to learn from the real-time Citi Bike data? How could you capture this over time?

---

---

---